In [6]:
import asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from renetti.ws.spiders.utils import parse_product_json_ld_from_page

In [7]:
url = "https://www.matrixfitness.com/uk/eng/strength/catalog"
base_url = "https://www.matrixfitness.com"

In [27]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                await page.goto(url=url)
                await asyncio.sleep(5)

                urls = []
                button = page.locator("#CybotCookiebotDialogBodyLevelButtonLevelOptinAllowallSelection")
                if await button.is_visible():
                    await button.scroll_into_view_if_needed(timeout=3000)
                    await button.click()
                    await asyncio.sleep(5)

                while True:
                    html = await page.content()
                    soup = BeautifulSoup(markup=html, features="html.parser")
                    urls += [
                        f"{base_url}{a.get('href')}"
                        for a in soup.find("matrix-catalog-grid").findAll("a")
                        if a.get("href") is not None and "onyx.matrixfitness.com" not in a.get("href")
                    ]

                    try:
                        button = page.locator("button.btn-primary")
                        if await button.is_visible():
                            await button.scroll_into_view_if_needed(timeout=3000)
                            await button.click()
                            await asyncio.sleep(5)
                        else:
                            break
                    except Exception as e:
                        break

res = list(set(urls))

In [28]:
len(res)

176

In [4]:
url = "https://www.matrixfitness.com/uk/eng/group-training/cardio/total-body-cycle"

In [5]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                await page.goto(url=url)
                await page.wait_for_selector('div.product-gallery')
                html = await page.content()
                soup = BeautifulSoup(markup=html, features="html.parser")
                res = parse_product_json_ld_from_page(soup=soup)

TimeoutError: Page.wait_for_selector: Timeout 30000ms exceeded.
Call log:
waiting for locator("div.product-gallery") to be visible


In [32]:
res

{'name': 'Magnum Back Extension Bench',
 'image_links': [],
 'mpn': None,
 'description': 'The Magnum Back Extension Bench is designed for improving abdominal, back and other core muscles. Large thigh pads and a skid-resistant foot platform provide added stability and comfort. Additionally, the bench is angled for easy entry and exit.',
 'brands': ['Matrix Fitness'],
 'categories': [],
 'skus': ['MG-A52']}